In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import time 
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
import plotly.graph_objects as go
import re 
import nltk  
nltk.download('stopwords') 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from collections import Counter
import cufflinks as cf
cf.go_offline()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission =  pd.read_csv("sample_submission.csv")

In [0]:
def full_clean(text):
    pattern = re.compile('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+')
    no_html= pattern.sub('',text)
    text = re.sub('[^a-zA-Z]', ' ', no_html)
    text = re.sub('[^a-zA-Z]', ' ', text)  
    text = text.lower() 
    text = text.split()
    text = [w for w in text if not w in set(stopwords.words('english'))]
    text = ' '.join(text)
    return text  


In [0]:
#cleaning the tweets

train['text']=train['text'].apply(lambda x : full_clean(x))
test['text']=test['text'].apply(lambda x : full_clean(x))



In [0]:
def counter_word (text):
    count = Counter()
    for i in text.values:
        for word in i.split():
            count[word] += 1
    return count
text_values=train["text"]
counter = counter_word(text_values)

In [0]:
#needed these vals to be global

oov_tok = "<XXX>"
vocab_size = len(counter)
trunc_type='post'
padding_type='post'
training_size = 6090
max_length = 20
training_sentences = train.text[0:training_size]
training_labels = train.target[0:training_size]

testing_sentences = train.text[training_size:]
testing_labels = train.target[training_size:]

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)

word_index = tokenizer.word_index

training_sequences = tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [9]:
#trains the model and returns list of predicted values
def ret_predictions_model():
  embedding_dim = 32
  num_epochs = 10
  model = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
  tf.keras.layers.Dense(14, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid') 
  ])
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  history = model.fit(training_padded, training_labels, epochs=num_epochs, validation_data=(testing_padded, testing_labels))
  predictions = model.predict_classes(testing_padded)
  predictions= (predictions > 0.5).astype(int)
  return predictions.ravel().tolist()


#returns list of ground truths
def ret_ground_truth():
  return testing_labels.ravel().tolist()

print(ret_predictions_model())
print(ret_ground_truth())




Epoch 1/10
191/191 [==============================] - 6s 33ms/step - loss: 0.5563 - accuracy: 0.7020 - val_loss: 0.4559 - val_accuracy: 0.7859
Epoch 2/10
191/191 [==============================] - 5s 27ms/step - loss: 0.2957 - accuracy: 0.8878 - val_loss: 0.5410 - val_accuracy: 0.7544
Epoch 3/10
191/191 [==============================] - 5s 28ms/step - loss: 0.1708 - accuracy: 0.9411 - val_loss: 0.5996 - val_accuracy: 0.7393
Epoch 4/10
191/191 [==============================] - 5s 29ms/step - loss: 0.1156 - accuracy: 0.9640 - val_loss: 0.6974 - val_accuracy: 0.7551
Epoch 5/10
191/191 [==============================] - 6s 29ms/step - loss: 0.0914 - accuracy: 0.9708 - val_loss: 0.6048 - val_accuracy: 0.7676
Epoch 6/10
191/191 [==============================] - 6s 29ms/step - loss: 0.0847 - accuracy: 0.9727 - val_loss: 0.7560 - val_accuracy: 0.7321
Epoch 7/10
191/191 [==============================] - 5s 28ms/step - loss: 0.0688 - accuracy: 0.9752 - val_loss: 0.8457 - val_accuracy: 0.7347